# Multi-Curve Framework in Interest Rate Modeling

The multi-curve framework has become the industry standard for pricing interest rate derivatives and managing risk. This approach emerged after the 2008 financial crisis, as market realities revealed significant differences between various reference rates. In this note, we discuss the theory behind multi-curve modeling, why it is used, and its mathematical formulation.

---

## 1. Why Use a Multi-Curve Framework?

Traditionally, a single yield curve was used for both discounting cash flows and forecasting future rates. However, market conditions revealed that:

- **Different Rates for Different Purposes:**  
  - The **Overnight Indexed Swap (OIS)** curve better reflects the risk-free (or collateralized) funding rate and is used for discounting cash flows.
  - **LIBOR, EURIBOR, or other interbank rates** are used for forecasting future floating rates.
  
- **Basis Spreads:**  
  The spreads (or differences) between these rates, known as basis spreads, are non-zero and vary across tenors. Ignoring these differences can lead to mispricing of instruments.

Thus, a **multi-curve framework** separates:
- The **discount curve** (often built from OIS rates)
- The **forward (forecast) curve(s)** (built from LIBOR, EURIBOR, etc.)

This separation provides a more realistic and accurate model for derivative pricing and risk management.

---

## 2. Mathematical Formulation

### 2.1 Discount Curve

The discount factor \( P(0,T) \) for cash flow at time \( T \) is computed using the continuously compounded risk-free rate \( r(T) \):

$$
P(0,T) = e^{-r(T) \, T}
$$

This discount curve is typically bootstrapped from market instruments such as OIS.

---

### 2.2 Forward Curve

The forward rate \( F(t, T) \) for a given period \([T, T+\Delta]\) is derived from the forward curve. If \( P^{\text{Frd}}(0,T) \) denotes the discount factor from the forward curve, then the forward rate is given by:

$$
F(0; T, T+\Delta) = \frac{1}{\Delta} \left( \frac{P^{\text{Frd}}(0,T)}{P^{\text{Frd}}(0,T+\Delta)} - 1 \right)
$$

Here, \(\Delta\) is the accrual period (e.g., 0.25 for a 3-month period).

---

### 2.3 Basis Spread

The basis spread \( S(T) \) accounts for the difference between the forecasting rate (e.g., LIBOR) and the risk-free (OIS) rate. It can be modeled as:

$$
F_{\text{LIBOR}}(0; T, T+\Delta) = F_{\text{OIS}}(0; T, T+\Delta) + S(T)
$$

Where:
- \( F_{\text{LIBOR}}(0; T, T+\Delta) \) is the forward rate implied by LIBOR.
- \( F_{\text{OIS}}(0; T, T+\Delta) \) is the forward rate implied by the OIS discount curve.
- \( S(T) \) is the basis spread at time \( T \).

---

### 2.4 Pricing Derivatives in a Multi-Curve Environment

Consider an interest rate swap where a fixed rate \( K \) is exchanged for a floating rate. In a multi-curve setting, the floating rate is forecast using the forward curve, and all cash flows are discounted using the OIS (discount) curve.

The present value \( V_{\text{swap}} \) of a payer swap can be expressed as:

$$
V_{\text{swap}} = \text{notional} \times \sum_{j=1}^{N} \tau_j \, P^{\text{Discount}}(0, T_j) \left( F(0; T_{j-1}, T_j) - K \right)
$$

where:
- \( \tau_j \) is the accrual factor for period \( j \),
- \( P^{\text{Discount}}(0, T_j) \) is the discount factor from the OIS curve,
- \( F(0; T_{j-1}, T_j) \) is the forward rate from the forward curve,
- \( K \) is the fixed rate (strike).

This formula ensures that the forecasting and discounting processes are handled by the appropriate curves.

---

## 3. Implementation Steps

1. **Bootstrap the Discount Curve:**  
   - Use OIS instruments to calibrate the discount curve.
   - Compute \( P^{\text{Discount}}(0,T) = e^{-r(T) T} \).

2. **Construct the Forward Curve:**  
   - Use market instruments (e.g., LIBOR-based swaps) to build the forward curve.
   - Derive forward rates using the relationship:
     
     $$
     F(0; T, T+\Delta) = \frac{1}{\Delta} \left( \frac{P^{\text{Frd}}(0,T)}{P^{\text{Frd}}(0,T+\Delta)} - 1 \right)
     $$

3. **Calibrate Basis Spreads:**  
   - Compare the forward rates from the forward curve with those implied by the discount curve.
   - Adjust for the basis spread \( S(T) \).

4. **Price Derivatives:**  
   - For each cash flow, forecast the floating rate using the forward curve.
   - Discount the cash flow using the discount curve.
   - Sum the present values to get the derivative price.

---

## 4. Benefits of the Multi-Curve Framework

- **Accuracy:**  
  Reflects the market-observed differences between various rates (e.g., OIS vs. LIBOR).
- **Risk Management:**  
  Allows for better hedging by modeling basis risk explicitly.
- **Market Consistency:**  
  Provides a pricing methodology that is consistent with the collateralization and funding practices seen in the market.

---

## Summary of Key Formulas

1. **Discount Factor:**

   $$
   P(0,T) = e^{-r(T) \, T}
   $$

2. **Forward Rate:**

   $$
   F(0; T, T+\Delta) = \frac{1}{\Delta} \left( \frac{P^{\text{Frd}}(0,T)}{P^{\text{Frd}}(0,T+\Delta)} - 1 \right)
   $$

3. **Basis Spread:**

   $$
   F_{\text{LIBOR}}(0; T, T+\Delta) = F_{\text{OIS}}(0; T, T+\Delta) + S(T)
   $$

4. **Swap Price (Multi-Curve):**

   $$
   V_{\text{swap}} = \text{notional} \times \sum_{j=1}^{N} \tau_j \, P^{\text{Discount}}(0, T_j) \left( F(0; T_{j-1}, T_j) - K \right)
   $$

---

This multi-curve framework is essential for modern interest rate modeling, providing improved accuracy and consistency in pricing, hedging, and risk management.


In [ ]:
import numpy as np
import enum
from copy import deepcopy
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev, interp1d


class OptionTypeSwap(enum.Enum):
    RECEIVER = 1.0
    PAYER = -1.0


def IRSwap(CP,notional,K,t,Ti,Tm,n,P0T):
    # CP- payer of receiver
    # n- notional
    # K- strike
    # t- today's date
    # Ti- beginning of the swap
    # Tm- end of Swap
    # n- number of dates payments between Ti and Tm
    # r_t -interest rate at time t
    ti_grid = np.linspace(Ti,Tm,int(n))
    tau = ti_grid[1]- ti_grid[0]

    temp= 0.0

    for (idx,ti) in enumerate(ti_grid):
        if idx>0:
            temp = temp + tau * P0T(ti)

    P_t_Ti = P0T(Ti)
    P_t_Tm = P0T(Tm)

    if CP==OptionTypeSwap.PAYER:
        swap = (P_t_Ti - P_t_Tm) - K * temp
    elif CP==OptionTypeSwap.RECEIVER:
        swap = K * temp - (P_t_Ti - P_t_Tm)

    return swap * notional

def IRSwapMultiCurve(CP,notional,K,t,Ti,Tm,n,P0T,P0TFrd):
    # CP- payer of receiver
    # n- notional
    # K- strike
    # t- today's date
    # Ti- beginning of the swap
    # Tm- end of Swap
    # n- number of dates payments between Ti and Tm
    # r_t -interest rate at time t
    ti_grid=np.linspace(Ti,Tm,int(n))
    tau=ti_grid[1]-ti_grid[0]
    swap=0.0
    for (idx,ti) in enumerate(ti_grid):
        if idx>0:
            L_frwd=1.0/tau*(P0TFrd(ti_grid[idx-1])-P0TFrd(ti_grid[idx]))/P0TFrd(ti_grid[idx])
            swap=swap+tau*P0T(ti_grid[idx])*(L_frwd-K)
    return swap*notional

def P0TModel(t,ti,ri,method):
    rInterp = method(ti,ri)
    r = rInterp(t)
    return np.exp(-r*t)

def YieldCurve(instruments, maturities, r0, method, tol):
    r0 = deepcopy(r0)
    ri = MultivariateNewtonRaphson(r0, maturities, instruments, method, tol=tol)
    return ri

def MultivariateNewtonRaphson(ri, ti, instruments, method, tol):
    err = 10e10
    idx = 0
    while err > tol:
        idx = idx +1
        values = EvaluateInstruments(ti,ri,instruments,method)
        J = Jacobian(ti,ri, instruments, method)
        J_inv = np.linalg.inv(J)
        err = - np.dot(J_inv, values)
        ri[0:] = ri[0:] + err
        err = np.linalg.norm(err)
        print('index in the loop is',idx,' Error is ', err)
    return ri

def Jacobian(ti, ri, instruments, method):
    eps = 1e-05
    swap_num = len(ti)
    J = np.zeros([swap_num, swap_num])
    val = EvaluateInstruments(ti,ri,instruments,method)
    ri_up = deepcopy(ri)

    for j in range(0, len(ri)):
        ri_up[j] = ri[j] + eps
        val_up = EvaluateInstruments(ti,ri_up,instruments,method)
        ri_up[j] = ri[j]
        dv = (val_up - val) / eps
        J[:, j] = dv[:]
    return J

def EvaluateInstruments(ti,ri,instruments,method):
    P0Ttemp = lambda t: P0TModel(t,ti,ri,method)
    val = np.zeros(len(instruments))
    for i in range(0,len(instruments)):
        val[i] = instruments[i](P0Ttemp)
    return val

def linear_interpolation(ti,ri):
    interpolator = lambda t: np.interp(t, ti, ri)
    return interpolator

def spline_interpolate(ti,ri):
    interpolator = splrep(ti, ri, s=0.01)
    interp = lambda t: splev(t,interpolator)
    return interp

def scipy_1d_interpolate(ti, ri):
    interpolator = lambda t: interp1d(ti, ri, kind='quadratic')(t)
    return interpolator

def mainCode():

    # Convergence tolerance
    tol = 1.0e-15
    # Initial guess for the spine points
    r0 = np.array([0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01])
    # Interpolation method
    method = linear_interpolation

    K   = np.array([0.04/100.0,	0.16/100.0,	0.31/100.0,	0.81/100.0,	1.28/100.0,	1.62/100.0,	2.22/100.0,	2.30/100.0])
    mat = np.array([1.0,2.0,3.0,5.0,7.0,10.0,20.0,30.0])

    #                   IRSwap(CP,           notional,K,   t,   Ti,Tm,   n,P0T)
    swap1 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[0],0.0,0.0,mat[0],4*mat[0],P0T)
    swap2 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[1],0.0,0.0,mat[1],5*mat[1],P0T)
    swap3 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[2],0.0,0.0,mat[2],6*mat[2],P0T)
    swap4 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[3],0.0,0.0,mat[3],7*mat[3],P0T)
    swap5 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[4],0.0,0.0,mat[4],8*mat[4],P0T)
    swap6 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[5],0.0,0.0,mat[5],9*mat[5],P0T)
    swap7 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[6],0.0,0.0,mat[6],10*mat[6],P0T)
    swap8 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,K[7],0.0,0.0,mat[7],11*mat[7],P0T)
    instruments = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8]

    # determine optimal spine points
    ri = YieldCurve(instruments, mat, r0, method, tol)
    print('\n Spine points are',ri,'\n')

    # Build a ZCB-curve/yield curve from the spine points
    P0T_Initial = lambda t: P0TModel(t,mat,r0,method)
    P0T = lambda t: P0TModel(t,mat,ri,method)

    swapsModel = np.zeros(len(instruments))
    swapsInitial = np.zeros(len(instruments))
    for i in range(0,len(instruments)):
        swapsModel[i] = instruments[i](P0T)
        swapsInitial[i] = instruments[i](P0T_Initial)

    print('Prices for Pas Swaps (initial) = ',swapsInitial,'\n')
    print('Prices for Par Swaps = ',swapsModel,'\n')

    # multie curve extention
    P0TFrd=deepcopy(P0T)
    Ktest=0.2
    swap1 = lambda P0T: IRSwap(OptionTypeSwap.PAYER,1,Ktest,0.0,0.0,mat[0],4*mat[0],P0T)
    swap1MC = lambda P0T: IRSwapMultiCurve(OptionTypeSwap.PAYER,1,Ktest,0.0,0.0,mat[0],4*mat[0],P0T,P0TFrd)
    print('Sanity check: swap1 = {0}, swap2 = {1}'.format(swap1(P0T),swap1MC(P0T)))

    # forward curve seeting
    r0Frwd = np.array([0.01, 0.01, 0.01, 0.01])
    KFrwd   = np.array([0.09/100.0,	0.26/100.0,	0.37/100.0,	1.91/100.0])
    matFrwd = np.array([1.0, 2.0, 3.0, 5.0])

    P0TDiscount=lambda t: P0TModel(t,mat,ri,method)
    swap1Frwd = lambda P0TFrwd: IRSwapMultiCurve(OptionTypeSwap.PAYER,1,KFrwd[0],0.0,0.0,matFrwd[0],4*matFrwd[0],P0TDiscount,P0TFrwd)
    swap2Frwd = lambda P0TFrwd: IRSwapMultiCurve(OptionTypeSwap.PAYER,1,KFrwd[1],0.0,0.0,matFrwd[1],5*matFrwd[1],P0TDiscount,P0TFrwd)
    swap3Frwd = lambda P0TFrwd: IRSwapMultiCurve(OptionTypeSwap.PAYER,1,KFrwd[2],0.0,0.0,matFrwd[2],6*matFrwd[2],P0TDiscount,P0TFrwd)
    swap4Frwd = lambda P0TFrwd: IRSwapMultiCurve(OptionTypeSwap.PAYER,1,KFrwd[3],0.0,0.0,matFrwd[3],7*matFrwd[3],P0TDiscount,P0TFrwd)
    instrumentsFrwd = [swap1Frwd,swap2Frwd,swap3Frwd,swap4Frwd]

    riFrwd=YieldCurve(instrumentsFrwd, matFrwd, r0Frwd, method, tol)
    print('\n Frwd Spine points are',riFrwd,'\n')
    # building yeild curve from frwd
    P0TFrwd_Initial=lambda t:P0TModel(t, matFrwd,r0Frwd, method)
    P0TFrwd         = lambda t: P0TModel(t,matFrwd,riFrwd,method)
    swapsModelFrwd   = np.zeros(len(instrumentsFrwd))
    swapsInitialFrwd = np.zeros(len(instrumentsFrwd))

    for i in range(0,len(instrumentsFrwd)):
        swapsModelFrwd[i]=instrumentsFrwd[i](P0TFrwd)
        swapsInitialFrwd[i] = instrumentsFrwd[i](P0TFrwd_Initial)

    print(swap1Frwd(P0TFrwd))
    t = np.linspace(0,10,100)
    plt.figure()
    plt.plot(t,P0TDiscount(t),'--r')
    plt.plot(t,P0TFrwd(t),'-b')
    plt.legend(['discount','forecast'])



mainCode()
